In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
corpus_name = "cornell movie-dialogs corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "movie_lines.txt"))

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


In [3]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']] = lineObj
    return lines


# Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # Extract fields
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
            utterance_id_pattern = re.compile('L[0-9]+')
            lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
            # Reassemble lines
            convObj["lines"] = []
            for lineId in lineIds:
                convObj["lines"].append(lines[lineId])
            conversations.append(convObj)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

In [4]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# Load lines and process conversations
print("\nProcessing corpus...")
lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
print("\nLoading conversations...")
conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
                                  lines, MOVIE_CONVERSATIONS_FIELDS)

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines from file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister. 

In [5]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [6]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [7]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


In [8]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[   7,   50,   50, 5265,    5],
        [ 293,   92,   37,   66,   37],
        [ 117,    7,   70,   25, 7012],
        [1600,  278, 2809,  387,    6],
        [  83,  159,  188, 5219,    2],
        [  80,    6,    6,   66,    0],
        [   7,    2,    2,    2,    0],
        [   6,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([9, 7, 7, 7, 5])
target_variable: tensor([[ 50, 247,   4, 124,  96],
        [ 47, 117,   2,   4, 159],
        [  7,   7,   0,   2,   4],
        [118, 614,   0,   0, 169],
        [ 40,   6,   0,   0, 386],
        [809,   2,   0,   0,   4],
        [  6,   0,   0,   0,   2],
        [  2,   0,   0,   0,   0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True],
        [ True,  True, False, False,  True],
        [ True,  True, False, False,  True],
        [ True,  True, False, False,  True],
        [ True, 

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [10]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [11]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [12]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [13]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [14]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [15]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [16]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [17]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [18]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 5000
print_every = 1
save_every = 10

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.9668
Iteration: 2; Percent complete: 0.0%; Average loss: 8.8580
Iteration: 3; Percent complete: 0.0%; Average loss: 8.6863
Iteration: 4; Percent complete: 0.1%; Average loss: 8.3850
Iteration: 5; Percent complete: 0.1%; Average loss: 8.0719
Iteration: 6; Percent complete: 0.1%; Average loss: 7.6488
Iteration: 7; Percent complete: 0.1%; Average loss: 7.1346
Iteration: 8; Percent complete: 0.1%; Average loss: 7.0554
Iteration: 9; Percent complete: 0.1%; Average loss: 7.0917
Iteration: 10; Percent complete: 0.1%; Average loss: 6.7453
Iteration: 11; Percent complete: 0.1%; Average loss: 6.2819
Iteration: 12; Percent complete: 0.1%; Average loss: 5.7573
Iteration: 13; Percent complete: 0.2%; Average loss: 5.7414
Iteration: 14; Percent complete: 0.2%; Average loss: 5.5827
Iteration: 15; Percent complete: 0.2%; Average loss: 5.6254
Iteration: 16; Percent complete: 0.2%

Iteration: 135; Percent complete: 1.7%; Average loss: 4.4787
Iteration: 136; Percent complete: 1.7%; Average loss: 4.1856
Iteration: 137; Percent complete: 1.7%; Average loss: 4.0574
Iteration: 138; Percent complete: 1.7%; Average loss: 4.4978
Iteration: 139; Percent complete: 1.7%; Average loss: 4.4582
Iteration: 140; Percent complete: 1.8%; Average loss: 4.2675
Iteration: 141; Percent complete: 1.8%; Average loss: 4.2926
Iteration: 142; Percent complete: 1.8%; Average loss: 4.3853
Iteration: 143; Percent complete: 1.8%; Average loss: 4.3373
Iteration: 144; Percent complete: 1.8%; Average loss: 4.4792
Iteration: 145; Percent complete: 1.8%; Average loss: 4.2477
Iteration: 146; Percent complete: 1.8%; Average loss: 4.2420
Iteration: 147; Percent complete: 1.8%; Average loss: 4.1141
Iteration: 148; Percent complete: 1.8%; Average loss: 4.1578
Iteration: 149; Percent complete: 1.9%; Average loss: 4.2840
Iteration: 150; Percent complete: 1.9%; Average loss: 3.9797
Iteration: 151; Percent 

Iteration: 269; Percent complete: 3.4%; Average loss: 4.0805
Iteration: 270; Percent complete: 3.4%; Average loss: 3.8016
Iteration: 271; Percent complete: 3.4%; Average loss: 3.9847
Iteration: 272; Percent complete: 3.4%; Average loss: 4.0467
Iteration: 273; Percent complete: 3.4%; Average loss: 3.5553
Iteration: 274; Percent complete: 3.4%; Average loss: 4.0981
Iteration: 275; Percent complete: 3.4%; Average loss: 3.6605
Iteration: 276; Percent complete: 3.5%; Average loss: 3.7374
Iteration: 277; Percent complete: 3.5%; Average loss: 3.9680
Iteration: 278; Percent complete: 3.5%; Average loss: 3.8967
Iteration: 279; Percent complete: 3.5%; Average loss: 4.0041
Iteration: 280; Percent complete: 3.5%; Average loss: 4.0447
Iteration: 281; Percent complete: 3.5%; Average loss: 3.9627
Iteration: 282; Percent complete: 3.5%; Average loss: 4.0129
Iteration: 283; Percent complete: 3.5%; Average loss: 3.8955
Iteration: 284; Percent complete: 3.5%; Average loss: 3.9259
Iteration: 285; Percent 

Iteration: 403; Percent complete: 5.0%; Average loss: 3.6437
Iteration: 404; Percent complete: 5.1%; Average loss: 3.9251
Iteration: 405; Percent complete: 5.1%; Average loss: 3.5936
Iteration: 406; Percent complete: 5.1%; Average loss: 3.5921
Iteration: 407; Percent complete: 5.1%; Average loss: 3.7187
Iteration: 408; Percent complete: 5.1%; Average loss: 3.9158
Iteration: 409; Percent complete: 5.1%; Average loss: 3.9024
Iteration: 410; Percent complete: 5.1%; Average loss: 3.9438
Iteration: 411; Percent complete: 5.1%; Average loss: 3.7298
Iteration: 412; Percent complete: 5.1%; Average loss: 3.8320
Iteration: 413; Percent complete: 5.2%; Average loss: 3.8702
Iteration: 414; Percent complete: 5.2%; Average loss: 3.8568
Iteration: 415; Percent complete: 5.2%; Average loss: 4.0182
Iteration: 416; Percent complete: 5.2%; Average loss: 3.4878
Iteration: 417; Percent complete: 5.2%; Average loss: 3.8194
Iteration: 418; Percent complete: 5.2%; Average loss: 3.4937
Iteration: 419; Percent 

Iteration: 537; Percent complete: 6.7%; Average loss: 3.6166
Iteration: 538; Percent complete: 6.7%; Average loss: 3.5709
Iteration: 539; Percent complete: 6.7%; Average loss: 3.7895
Iteration: 540; Percent complete: 6.8%; Average loss: 3.6898
Iteration: 541; Percent complete: 6.8%; Average loss: 3.5969
Iteration: 542; Percent complete: 6.8%; Average loss: 3.7900
Iteration: 543; Percent complete: 6.8%; Average loss: 3.6825
Iteration: 544; Percent complete: 6.8%; Average loss: 3.8774
Iteration: 545; Percent complete: 6.8%; Average loss: 3.7340
Iteration: 546; Percent complete: 6.8%; Average loss: 3.7236
Iteration: 547; Percent complete: 6.8%; Average loss: 3.6773
Iteration: 548; Percent complete: 6.9%; Average loss: 3.3714
Iteration: 549; Percent complete: 6.9%; Average loss: 3.4250
Iteration: 550; Percent complete: 6.9%; Average loss: 3.7428
Iteration: 551; Percent complete: 6.9%; Average loss: 3.8166
Iteration: 552; Percent complete: 6.9%; Average loss: 3.5578
Iteration: 553; Percent 

...

Iteration: 7438; Percent complete: 93.0%; Average loss: 1.8592
Iteration: 7439; Percent complete: 93.0%; Average loss: 2.0739
Iteration: 7440; Percent complete: 93.0%; Average loss: 1.9226
Iteration: 7441; Percent complete: 93.0%; Average loss: 1.8163
Iteration: 7442; Percent complete: 93.0%; Average loss: 1.8437
Iteration: 7443; Percent complete: 93.0%; Average loss: 1.9935
Iteration: 7444; Percent complete: 93.0%; Average loss: 1.9780
Iteration: 7445; Percent complete: 93.1%; Average loss: 1.8745
Iteration: 7446; Percent complete: 93.1%; Average loss: 1.9322
Iteration: 7447; Percent complete: 93.1%; Average loss: 1.7934
Iteration: 7448; Percent complete: 93.1%; Average loss: 1.8567
Iteration: 7449; Percent complete: 93.1%; Average loss: 1.8119
Iteration: 7450; Percent complete: 93.1%; Average loss: 1.8406
Iteration: 7451; Percent complete: 93.1%; Average loss: 1.9373
Iteration: 7452; Percent complete: 93.2%; Average loss: 1.7027
Iteration: 7453; Percent complete: 93.2%; Average loss:

Iteration: 7568; Percent complete: 94.6%; Average loss: 1.8483
Iteration: 7569; Percent complete: 94.6%; Average loss: 1.7039
Iteration: 7570; Percent complete: 94.6%; Average loss: 1.7658
Iteration: 7571; Percent complete: 94.6%; Average loss: 1.8010
Iteration: 7572; Percent complete: 94.7%; Average loss: 1.9574
Iteration: 7573; Percent complete: 94.7%; Average loss: 1.8410
Iteration: 7574; Percent complete: 94.7%; Average loss: 1.6941
Iteration: 7575; Percent complete: 94.7%; Average loss: 1.5884
Iteration: 7576; Percent complete: 94.7%; Average loss: 1.7020
Iteration: 7577; Percent complete: 94.7%; Average loss: 1.7421
Iteration: 7578; Percent complete: 94.7%; Average loss: 1.8022
Iteration: 7579; Percent complete: 94.7%; Average loss: 1.5806
Iteration: 7580; Percent complete: 94.8%; Average loss: 1.8007
Iteration: 7581; Percent complete: 94.8%; Average loss: 1.8354
Iteration: 7582; Percent complete: 94.8%; Average loss: 1.9004
Iteration: 7583; Percent complete: 94.8%; Average loss:

Iteration: 7698; Percent complete: 96.2%; Average loss: 1.6561
Iteration: 7699; Percent complete: 96.2%; Average loss: 1.8179
Iteration: 7700; Percent complete: 96.2%; Average loss: 1.6561
Iteration: 7701; Percent complete: 96.3%; Average loss: 1.7838
Iteration: 7702; Percent complete: 96.3%; Average loss: 1.7182
Iteration: 7703; Percent complete: 96.3%; Average loss: 1.7137
Iteration: 7704; Percent complete: 96.3%; Average loss: 1.8262
Iteration: 7705; Percent complete: 96.3%; Average loss: 1.6646
Iteration: 7706; Percent complete: 96.3%; Average loss: 1.6425
Iteration: 7707; Percent complete: 96.3%; Average loss: 1.7839
Iteration: 7708; Percent complete: 96.4%; Average loss: 1.5396
Iteration: 7709; Percent complete: 96.4%; Average loss: 1.5945
Iteration: 7710; Percent complete: 96.4%; Average loss: 1.6727
Iteration: 7711; Percent complete: 96.4%; Average loss: 1.7424
Iteration: 7712; Percent complete: 96.4%; Average loss: 1.7270
Iteration: 7713; Percent complete: 96.4%; Average loss:

Iteration: 7828; Percent complete: 97.9%; Average loss: 1.7987
Iteration: 7829; Percent complete: 97.9%; Average loss: 1.6572
Iteration: 7830; Percent complete: 97.9%; Average loss: 1.8683
Iteration: 7831; Percent complete: 97.9%; Average loss: 1.6949
Iteration: 7832; Percent complete: 97.9%; Average loss: 1.7341
Iteration: 7833; Percent complete: 97.9%; Average loss: 1.9457
Iteration: 7834; Percent complete: 97.9%; Average loss: 1.9890
Iteration: 7835; Percent complete: 97.9%; Average loss: 1.5607
Iteration: 7836; Percent complete: 98.0%; Average loss: 1.7938
Iteration: 7837; Percent complete: 98.0%; Average loss: 1.7011
Iteration: 7838; Percent complete: 98.0%; Average loss: 1.7137
Iteration: 7839; Percent complete: 98.0%; Average loss: 1.5797
Iteration: 7840; Percent complete: 98.0%; Average loss: 1.6064
Iteration: 7841; Percent complete: 98.0%; Average loss: 2.0152
Iteration: 7842; Percent complete: 98.0%; Average loss: 2.0119
Iteration: 7843; Percent complete: 98.0%; Average loss:

Iteration: 7958; Percent complete: 99.5%; Average loss: 1.7233
Iteration: 7959; Percent complete: 99.5%; Average loss: 1.8722
Iteration: 7960; Percent complete: 99.5%; Average loss: 1.7919
Iteration: 7961; Percent complete: 99.5%; Average loss: 1.6230
Iteration: 7962; Percent complete: 99.5%; Average loss: 1.8255
Iteration: 7963; Percent complete: 99.5%; Average loss: 1.7774
Iteration: 7964; Percent complete: 99.6%; Average loss: 1.7095
Iteration: 7965; Percent complete: 99.6%; Average loss: 1.8706
Iteration: 7966; Percent complete: 99.6%; Average loss: 1.8479
Iteration: 7967; Percent complete: 99.6%; Average loss: 1.6382
Iteration: 7968; Percent complete: 99.6%; Average loss: 1.5446
Iteration: 7969; Percent complete: 99.6%; Average loss: 1.6740
Iteration: 7970; Percent complete: 99.6%; Average loss: 1.5991
Iteration: 7971; Percent complete: 99.6%; Average loss: 1.6972
Iteration: 7972; Percent complete: 99.7%; Average loss: 1.7336
Iteration: 7973; Percent complete: 99.7%; Average loss:

In [ ]:
encoder.eval()
decoder.eval()
searcher = GreedySearchDecoder(encoder, decoder)
evaluateInput(encoder, decoder, searcher, voc)

> Hello
Bot: hello . there . there . there
> how are you?
Bot: fine . in a minute . there
> what are yo saying?
Bot: i m looking for the police . !
> why?
Bot: because i have to . to him .
> i am stupid
Bot: yes you are . you re mad .
> i created you
Bot: yes sir . it ! it
> you are fighting with me?
Bot: yes . ? ? ? ?
> i will kill you?
Bot: when ? ? ? ? ?
> right now
Bot: gimme the rain straight . it !
> i will kill you now
Bot: then you ll go to jail up . there
> i won't
Bot: i m sorry . i m leaving .
> yes that's good
Bot: good . there s another drink there
> want to have sex?
Bot: no . here . there . ?
> are you a girl?
Bot: no . there . there . ?
> are you a boy?
Bot: whatiya mean ? ? ? ?
> bot
Error: Encountered unknown word.
> you are a robot
Error: Encountered unknown word.
> you are a boy
Bot: i m a thief . there .
> then go to jail
Bot: i love you . to them !
> i'm proposing to you
Bot: that s okay . on your mouth .
> i'm handsome
Bot: what ? ? ? ? ?
> i'm sexy
Bot: me too .